# Welcome to the Schema Validator

This notebook contains examples for how to use the schema validator library, highlights the motivations and current shortcomings

In [2]:
import pandera as pa
import sqlalchemy
import schema_validator
import pandas as pd
from testcontainers.postgres import PostgresContainer

# Pandera Basics

Pandera is a nifty 3rd-party library that can be used to make sure DataFrames satisfy a specified structure / schema. 

In [3]:
example_dataframe_schema = pa.DataFrameSchema({
    "Name": pa.Column(pa.String),
    "Age": pa.Column(pa.Int, checks=[pa.Check.less_than(100), pa.Check.greater_than(0)]),
    "Genre": pa.Column(pa.String, checks=pa.Check.isin(["Pop", "Rock", "Jazz"]))
})

valid_df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35], 'Genre': ["Pop", "Rock", "Jazz"]})

example_dataframe_schema.validate(valid_df)

,Name,Age,Genre
0,Alice,25,Pop
1,Bob,30,Rock
2,Charlie,35,Jazz


Pandera verifies the presence of columns, checks their dtypes, and enforces any other checks (nullability, greater/less than, allowed values, custom functions...)

If any checks fail, a `pa.errors.SchemaError` will be raised on the call to `validate`

Let's try a few examples that should fail...

In [4]:
# This should fail due to negative age
invalid_age_df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [-1, 30, 35], 'Genre': ["Pop", "Rock", "Jazz"]})

try:
    example_dataframe_schema.validate(invalid_age_df)
except pa.errors.SchemaError as e:
    print(e)

Column 'Age' failed element-wise validator number 1: greater_than(0) failure cases: -1


In [5]:
# This should fail due to genres outside of the allowed values 
invalid_genre_df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35], 'Genre': ["Soul", "Funk", "Classical"]})

try:
    example_dataframe_schema.validate(invalid_genre_df)
except pa.errors.SchemaError as e:
    print(e)

Column 'Genre' failed element-wise validator number 0: isin(['Pop', 'Rock', 'Jazz']) failure cases: Soul, Funk, Classical


In [6]:
# This should fail due to missing genre column
missing_column_df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35]})

try:
    example_dataframe_schema.validate(missing_column_df)
except pa.errors.SchemaError as e:
    print(e)

column 'Genre' not in dataframe. Columns in dataframe: ['Name', 'Age']


Pandera also allows us to specify a schema using pydantic-style syntax:

In [7]:
class ExampleDataFrameModel(pa.DataFrameModel):
    Age: int = pa.Field(ge=0, le=100)
    Name: str
    Genre: str = pa.Field(isin=["Pop", "Rock", "Jazz"])

ExampleDataFrameModel.validate(valid_df)

,Name,Age,Genre
0,Alice,25,Pop
1,Bob,30,Rock
2,Charlie,35,Jazz


## Pandera Custom Checks

Just to demonstrate how powerful Pandera checks can be, let's create a custom check that verifies every other letter in a string is "e":

In [8]:
def every_other_letter_is_e(string: str) -> bool:
    return all((char == "e") for char in string[::2])

valid_strings_with_e = ["e_e_e", "eeee", "eAeBe"]

custom_check_schema = pa.DataFrameSchema({
    "info": pa.Column(str, checks=pa.Check(every_other_letter_is_e, element_wise=True))
})

custom_valid_df = pd.DataFrame({'info': valid_strings_with_e})
custom_check_schema.validate(custom_valid_df)

,info
0,e_e_e
1,eeee
2,eAeBe


In [9]:
invalid_strings_with_e = ["this_doesnt_pass", "nor does this", "eeeeeeeeeeeee"]

custom_invalid_df = pd.DataFrame({'info': invalid_strings_with_e})

try:
    custom_check_schema.validate(custom_invalid_df)
except pa.errors.SchemaError as e:
    print(e)

Column 'info' failed element-wise validator number 0: <Check every_other_letter_is_e> failure cases: this_doesnt_pass, nor does this


# Applying Pandera to a Database

Pandera is designed to validate various types of DataFrames (pandera, polars, spark), but not tables in a Database.

However, some Pandera schemas would be useful to validate the structure of tables because they:
- Serve as a source of documentation that must stay in sync with the code, unlike long docstrings which can easily drift from reality
- Verify calculations are being done correctly, and avoids tricky issues around types that might otherwise go unnoticed
- Serve as a valuable scaffolding while refactoring code

To support this, I created the `schema_validator` library!

Let's walk through an example of using the schema_validator on the existing `valid_df` that we've defined above

In [10]:
# Spin up a Postgres database in a container
postgres = PostgresContainer("postgres:16")
postgres.start()
engine = sqlalchemy.create_engine(postgres.get_connection_url())

Pulling image postgres:16
Container started: 1e3f2e32c077
Waiting to be ready...
Waiting to be ready...
Waiting to be ready...


In [11]:
# Load the valid_df into the database in a table called valid_example
valid_df.to_sql("valid_example", engine, schema="public", index=False, if_exists="replace")

with engine.connect() as connection:
    valid_df_in_db = pd.read_sql("SELECT * FROM public.valid_example", connection)
valid_df_in_db

,Name,Age,Genre
0,Alice,25,Pop
1,Bob,30,Rock
2,Charlie,35,Jazz


To validate the table, we need to create an instance of the `schema_validator.Table` class which defines:
- The Pandera schema to use for validation
- The database schema where the table exists
- The name of the table

In [12]:
valid_table = schema_validator.Table(
    name="valid_example",
    db_schema="public",
    table_schema=example_dataframe_schema
)
valid_table.validate(engine)

[DEBUG: 2025-06-15 12:05:58 UTC - table.py:195 - _raise_errors_if_columns_dont_match()] Schema validation passed for table valid_example


public.valid_example

Let's create some tables that will not pass validation to see what happens

In [13]:
# The dataframe we previously defined with the missing "Genre" column
missing_column_df.to_sql("missing_column_table", engine, schema="public", index=False, if_exists="replace")
missing_column_table = schema_validator.Table(
    name="missing_column_table",
    db_schema="public",
    table_schema=example_dataframe_schema
)

try:
    missing_column_table.validate(engine)
except schema_validator.exceptions.SchemaValidationError as e:
    print(e)

Schema validation failed for table missing_column_table: Number of columns in Pandera schema (3) does not match number of columns in database (2).
Pandera columns: [Name, Age, Genre]
Database columns: [Name, Age]


In [14]:
wrong_type_df = pd.DataFrame({'Name': [True, False, True], 'Age': [1.1, 2.2, 3.3], 'Genre': [1, 2, 3]})
wrong_type_df.to_sql("wrong_type_table", engine, schema="public", index=False, if_exists="replace")

wrong_type_table = schema_validator.Table(
    name="wrong_type_table",
    db_schema="public",
    table_schema=example_dataframe_schema
)

try:
    wrong_type_table.validate(engine)
except schema_validator.exceptions.SchemaValidationError as e:
    print(e)

Schema validation failed for table wrong_type_table:
	- Column Name dtype in the database does not match the Pandera schema: Pandera dtype=<class 'schema_validator.unified_types.String'>, Database dtype=<class 'schema_validator.unified_types.Boolean'>
	- Column Age dtype in the database does not match the Pandera schema: Pandera dtype=<class 'schema_validator.unified_types.Integer'>, Database dtype=<class 'schema_validator.unified_types.Float'>
	- Column Genre dtype in the database does not match the Pandera schema: Pandera dtype=<class 'schema_validator.unified_types.String'>, Database dtype=<class 'schema_validator.unified_types.Integer'>


We can also check for whether a column allows null values.

By default, Pandera schemas assume columns DO allow null values, while database tables usually assume they don't.

To check for nullability, use the `check_nullable` argument to `schema_validator.Table.validate()`.

In [15]:
# Example using Pandera directly on a DataFrame

class NoNonesAllowedSchema(pa.DataFrameModel):
    column: str = pa.Field(nullable=False)

df_with_nones = pd.DataFrame({'column': [None, "a", "b"]})

try:
    NoNonesAllowedSchema.validate(df_with_nones)
except pa.errors.SchemaError as e:
    print(e)

non-nullable series 'column' contains null values:
0    None
Name: column, dtype: object


In [16]:
# Example using the schema_validator on the table
df_with_nones.to_sql("no_nones_allowed_table", engine, schema="public", index=False, if_exists="replace")

no_nones_allowed_table = schema_validator.Table(
    name="no_nones_allowed_table",
    db_schema="public",
    table_schema=NoNonesAllowedSchema
)

try:
    no_nones_allowed_table.validate(engine, check_nullable=True)
except schema_validator.exceptions.SchemaValidationError as e:
    print(e)

Schema validation failed for table no_nones_allowed_table:
	- Column column has nullable=False in the schema but nullable=True in the database


# How does it work?

Pandera and SQLAlchemy each have their own type systems.
Both type systems use inheritance to define more detailed type relations.

In [17]:
sqlalchemy_all_types = '\n'.join(sqlalchemy.types.__all__)
print(f"SQLAlchemy defines the following types: {sqlalchemy_all_types}")

SQLAlchemy defines the following types: TypeEngine
TypeDecorator
UserDefinedType
ExternalType
INT
CHAR
VARCHAR
NCHAR
NVARCHAR
TEXT
Text
FLOAT
NUMERIC
REAL
DECIMAL
TIMESTAMP
DATETIME
CLOB
BLOB
BINARY
VARBINARY
BOOLEAN
BIGINT
SMALLINT
INTEGER
DATE
TIME
TupleType
String
Integer
SmallInteger
BigInteger
Numeric
Float
DateTime
Date
Time
LargeBinary
Boolean
Unicode
Concatenable
UnicodeText
PickleType
Interval
Enum
Indexable
ARRAY
JSON


In [18]:
# Even though there are many types, there are only a handul of base types
# https://docs.sqlalchemy.org/en/20/core/type_basics.html
assert issubclass(sqlalchemy.TEXT, sqlalchemy.String)
assert issubclass(sqlalchemy.VARCHAR, sqlalchemy.String)
assert issubclass(sqlalchemy.BigInteger, sqlalchemy.Integer)
assert issubclass(sqlalchemy.FLOAT, sqlalchemy.Float)
assert issubclass(sqlalchemy.REAL, sqlalchemy.Float)
assert issubclass(sqlalchemy.TIMESTAMP, sqlalchemy.DateTime)

In [ ]:
# Pandera also has its own type system with similar inheritance
assert issubclass(pa.Int8, pa.Int)
assert issubclass(pa.Float128, pa.Float)
assert issubclass(pa.String, pa.DataType)

AssertionError: 

The `schema_validator` creates its own type system which defines relationships between Pandera and SQLAlchemy types, as well as Python native equivalents

In [ ]:
unified_types = [
    schema_validator.unified_types.String,
    schema_validator.unified_types.Boolean,
    schema_validator.unified_types.Integer,
    schema_validator.unified_types.Float,
    schema_validator.unified_types.DateTime,
    schema_validator.unified_types.Date,
    schema_validator.unified_types.Timedelta,
    schema_validator.unified_types.NoneType,
]

for utype in unified_types:
    print(f"Unified Type {utype} wraps {utype.pandera_dtype}, {utype.sqlalchemy_dtype} and {utype.native_type}")

Unified Type <class 'schema_validator.unified_types.String'> wraps <class 'pandera.dtypes.String'>, <class 'sqlalchemy.sql.sqltypes.String'> and <class 'str'>
Unified Type <class 'schema_validator.unified_types.Boolean'> wraps <class 'pandera.dtypes.Bool'>, <class 'sqlalchemy.sql.sqltypes.Boolean'> and <class 'bool'>
Unified Type <class 'schema_validator.unified_types.Integer'> wraps <class 'pandera.dtypes.Int'>, <class 'sqlalchemy.sql.sqltypes.Integer'> and <class 'int'>
Unified Type <class 'schema_validator.unified_types.Float'> wraps <class 'pandera.dtypes.Float'>, <class 'sqlalchemy.sql.sqltypes.Float'> and <class 'float'>
Unified Type <class 'schema_validator.unified_types.DateTime'> wraps <class 'pandera.dtypes.Timestamp'>, <class 'sqlalchemy.sql.sqltypes.DateTime'> and <class 'datetime.datetime'>
Unified Type <class 'schema_validator.unified_types.Date'> wraps <class 'pandera.dtypes.Date'>, <class 'sqlalchemy.sql.sqltypes.Date'> and <class 'datetime.date'>
Unified Type <class 's

The basic algorithm is:

- Use SQLAlchemy's `inspector` to parse the structure of a table
- Map the table's types to `UnifiedType`
- Map the Pandera schema's types to `UnifiedType`
- Compare unified types

# Limitations

- Most Pandera checks are not supported.  For now, only checking column existence, data type, and nullability is support.
    - More complex checks would require reading the whole table, which would be inefficient.
- SQLAlchemy's `inspector` does not work well for temporary tables, so we work around this by querying a single row and inferring types from the row.
- This approach is highly coupled to Pandera and SQLAlchemy.  If you use a different tool for connecting to a database, this won't work.